In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Environment Setup

In [ ]:
# Installing libraries
!pip install seaborn numpy pandas matplotlib nltk

In [ ]:
# pandas for maniplating, cleaning, and exploring data
import pandas as pd
# numerical computing in Python. arrays and performance optimization
import numpy as np
# plotting
import matplotlib.pyplot as plt
# built ontop of plt, heatmaps, boxplots, distribution plots
import seaborn as sns
# basic natural language processing. tokenizations
# stopwords, stemming, etc
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

## Split Datasets

In [ ]:
train = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/train.csv')
test = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/test.csv')
val = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/val.csv')

# notes from LB 
##  The machine learning workflow: 
<br> 1) Obtain Data 
<br> 2) Exploratory data analysis (Look how many missing values (in data rows) you have, look for outliers in length, do TF-IDF to see which words are most important to your dataset, make lots of charts matplotlib, seaborn) 
<br> 3) clean data (pandas) 
<br> 4) split data (80/20 train/test) 
<br> 5) model.train() 
<br> 6) model.test
<br> 7) SKLearn accuracy metrics
<br> 8 ) chart results


## Display first few rows of each dataset

### Training Data
Training data from the 2024 US Presidential election, sourcced from X (fromerly Twitter). It includes post text, candidate information, party affiliation, sentitment labels, retweet counts, and like. This data is ideal for training machine learning models to analyze public sentiment and trends.

In [ ]:
print('Train Dataset:', train.head())

### Testing Data
This file provides testing data from the 2024 US Presidential election. It contains post text, candidate details, party affilication, and engagement metrics. This dataset is used to evaluate the performance of models trains on the training data.

In [ ]:
print('\n Test Dataset:', test.head())

### Validation Data
This file contains validation data for the 2024 US Presidential election. It includes post content, candidates, party affiliation, and sentiment labels. Use this file to fine-tune and validate model accuracy and performance. 

In [ ]:
print('Validation Dataset:', val.head())

# Data Exploration

# Do TF-IDF to see most important words 

# 

Dataset overview for all 3 files

### `pandas.info()`
`.info()` is a pandas method that "prints a concise summary oof a DataFrame.
This method provides a concise summary about a DataFrame including data types, non-null values, and memeory usage for each column. <br>


In [ ]:
print('Train Dataset Info:')
train.info()

In [ ]:
print('Test Dataset Info: ')
test.info()

In [ ]:
print('Validation Dataset Info: ')
val.info()